<a href="https://colab.research.google.com/github/Bharasaurus/Chat-Bot/blob/main/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import json
import numpy as np
import string
import torch
import random
import tensorflow as tf
from tensorflow import keras
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
stemmer=PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)
def stem(word):
  return stemmer.stem(word.lower())
def Bag_of_Words(tokenized_sentence, all_words):
  sentence=[stem(word) for word in tokenized_sentence]
  bag=np.zeros(len(all_words),dtype=np.float32)
  for i,word in enumerate(all_words):
    if(word in sentence):
      bag[i]=1.0
  return bag

In [ ]:
with open("/content/drive/MyDrive/Deep Learning/Colab Notebooks/Chat Bot/Intents.json",'r') as obj:
  intents=json.load(obj)
print(intents)
all_words=[]
tags=[]
xy=[]

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [ ]:
for intent in intents["intents"]:
  tag=intent["tag"]
  tags.append(tag)
  for pattern in intent["patterns"]:
    all_words.extend(tokenize(pattern))
    xy.append((tokenize(pattern),tag))
all_words=[stem(word) for word in all_words if word not in string.punctuation]
all_words=sorted(set(all_words))
tags=sorted(set(tags))
print(xy)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thank', "'s", 'a', 'lot', '!'], 'thanks'), (['Which', 'items', 'do', 'you', 'have', '?'], 'items'), (['What', 'kinds', 'of', 'items', 'are', 'there', '?'], 'items'), (['What', 'do', 'you', 'sell', '?'], 'items'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Can', 'I', 'pay', 'with', 'Paypal', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['How', 'long', 'does', 'delivery', 'take', '?'], 'delivery'), (['How', 'long', 'does', 'shipping', 'take', '?'], 'delivery'), (['When', 'do', 'I', 'get', 'my', 'delivery', '?'], 'delivery'),

In [ ]:
X_train=[]
Y_train=[]
for (pattern_sentence, tag) in xy:
  bag=Bag_of_Words(pattern_sentence,all_words)
  X_train.append(bag)
  label=tags.index(tag)
  Y_train.append(label)
X_train=np.array(X_train)
X_train = X_train / np.max(X_train, axis=1, keepdims=True)
print(len(X_train))
Y_train=np.array(Y_train)
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=Y_train

  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.n_samples

#Hyperparameter

batch_size=8
hidden_size=8
output_size=len(tags)
input_size=len(X_train[0])
learning_rate=0.001
num_epochs=1000

dataset=ChatDataset()
train_loader=DataLoader(dataset=dataset, batch_size=batch_size,shuffle=True, num_workers=0)

88


In [ ]:
#Learning Model

class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(NeuralNet,self).__init__()
    self.l1=nn.Linear(input_size,hidden_size)
    self.l2=nn.Linear(hidden_size,hidden_size)
    self.l3=nn.Linear(hidden_size,hidden_size)
    self.l4=nn.Linear(hidden_size, num_classes)
    self.relu=nn.Sigmoid()

  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out)
    out=self.l2(out)
    out=self.relu(out)
    out=self.l3(out)
    out=self.relu(out)
    out=self.l4(out)
    return out

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=NeuralNet(input_size,hidden_size,output_size).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
for epoch in range(num_epochs):
  for (words,labels) in train_loader:
    words=words.to(device)
    labels=labels.to(device)
    #Forward
    outputs=model(words)
    loss=criterion(outputs,labels)
    #Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if(epoch+1)%100==0:
    print(f"epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}")

print(f"Final Loss - {loss.item():.4f}")

epoch 100/1000, loss=2.4564
epoch 200/1000, loss=1.6840
epoch 300/1000, loss=1.1307
epoch 400/1000, loss=0.8747
epoch 500/1000, loss=0.6221
epoch 600/1000, loss=0.3470
epoch 700/1000, loss=0.2782
epoch 800/1000, loss=0.1122
epoch 900/1000, loss=0.1424
epoch 1000/1000, loss=0.0695
Final Loss - 0.0695


In [67]:
# model = keras.Sequential([
#     keras.layers.Dense(128, input_shape=(len(X_train[0]),), activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(len(tags), activation='softmax')
# ])
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9)
# optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
# model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(X_train, Y_train, epochs=1000)

In [68]:
#Feed-Back Loop

def incremental_train(model, optimizer, criterion, new_patterns, new_labels, num_epochs=100):
    new_patterns = [Bag_of_Words(pattern, all_words) for pattern in new_patterns]
    new_labels = [tags.index(label) for label in new_labels]

    new_patterns = np.array(new_patterns)
    new_patterns = new_patterns / np.max(new_patterns, axis=1, keepdims=True)
    new_labels = np.array(new_labels)

    new_dataset = ChatDataset()
    new_dataset.x_data = new_patterns
    new_dataset.y_data = new_labels
    new_loader = DataLoader(dataset=new_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    for epoch in range(num_epochs):
        for (words, labels) in new_loader:
            words = words.to(device)
            labels = labels.to(device)
            # Forward
            outputs = model(words)
            loss = criterion(outputs, labels)
            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Incremental Training - epoch {epoch + 1}/{num_epochs}, loss={loss.item():.4f}")


In [72]:
bot_name="Sam"
print("Let's Chat! Type 'quit' to exit")
while True:
  sentence=input("You - ")
  if(sentence.lower()=="quit"):
    break
  sentence=tokenize(sentence)
  X=Bag_of_Words(sentence, all_words)
  X=X.reshape(1,X.shape[0])
  X=torch.from_numpy(X)
  X=X.to(device)
  output=model(X)
  _, predicted=torch.max(output,dim=1)
  tag=tags[predicted.item()]
  probs=torch.softmax(output, dim=1)
  prob=probs[0][predicted.item()]
  if prob.item() > 0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        response = random.choice(intent['responses'])
        print(f"{bot_name} - {response}")

        # Ask for user feedback
        user_feedback = input("Did this answer your question? (yes/no): ")
        if user_feedback.lower() == "no":
          comment = input("Please provide additional comments: ")
          # Update intents with new patterns
          new_patterns = tokenize(comment)
          intents["intents"].append({
                        "tag": "custom",
                        "patterns": new_patterns,
                        "responses": ["Thank you for your input. We'll improve our responses."]})
          incremental_train(model, optimizer, criterion, new_patterns, ["custom"], num_epochs=50)

  else:
        print(f"{bot_name}: Sorry, please repeat your prompt")


Let's Chat! Type 'quit' to exit
You - hello
Sam - Hi there, how can I help?
Did this answer your question? (yes/no): yes
You - jokes pls
Sam - What did the buffalo say when his son left for college? Bison.
Did this answer your question? (yes/no): yes
You - loll
Sam: Sorry, please repeat your prompt
You - bye
Sam - Bye! Come back again soon.
Did this answer your question? (yes/no): yes
You - quit
